In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sb
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from statsmodels.tsa.arima.model import ARIMA
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
import gc
import lightgbm as lgb
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
from sklearn.model_selection import GroupKFold
%matplotlib inline
print("Libraries imported")

Libraries imported


In [2]:
cc_train = pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
cc_test = pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
cc_sample = pd.read_csv('/kaggle/input/widsdatathon2023/sample_solution.csv')
label = pd.read_csv("/kaggle/input/pseudo-labeling/submission_pseudo_labeling.csv")

In [3]:
temp = cc_test.copy()
temp["contest-tmp2m-14d__tmp2m"] = label["contest-tmp2m-14d__tmp2m"]
cc_train = cc_train.append(temp.sample(frac=0.8)).reset_index(drop=True)

In [4]:
def reduce_mem_usage(dataframe, verbose=True):
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_memory = dataframe.memory_usage().sum() / 1024**2
  for col in dataframe.columns:
    col_type = dataframe[col].dtypes
    if col_type in numerics:
      c_min = dataframe[col].min()
      c_max = dataframe[col].max()
      if str(col_type)[:3] == 'int':
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
          dataframe[col] = dataframe[col].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
          dataframe[col] = dataframe[col].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
          dataframe[col] = dataframe[col].astype(np.int32)
        elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
          dataframe[col] = dataframe[col].astype(np.int64)
      else:
        if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
          dataframe[col] = dataframe[col].astype(np.float32)
        else:
          dataframe[col] = dataframe[col].astype(np.float64)
  end_memory = dataframe.memory_usage().sum() / 1024**2
  print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_memory, 100 * (start_memory - end_memory) / start_memory)) if verbose else print('Reduced to {:5.2f}'.format(end_memory))
  return dataframe

In [5]:
cc_train = reduce_mem_usage(cc_train)

Mem. usage decreased to 379.19 Mb (49.6% reduction)


In [6]:
month_to_season = {
      1: 0,
      2: 0,
      3: 1,
      4: 1,
      5: 1,
      6: 2,
      7: 2,
      8: 2, 
      9: 3, 
      10: 3,
      11: 3,
      12: 0
  }

In [7]:
def location_feature(train, test):
    # Reference: https://www.kaggle.com/code/flaviafelicioni/wids-2023-different-locations-train-test-solved
    scale = 14
    train.loc[:,'lat']=round(train.lat,scale)
    train.loc[:,'lon']=round(train.lon,scale)
    test.loc[:,'lat']=round(test.lat,scale)
    test.loc[:,'lon']=round(test.lon,scale)
    
    train_and_test = pd.concat([train, test], axis=0)
    train_and_test['loc_group'] = train_and_test.groupby(['lat', 'lon']).ngroup()
    print(f'{train_and_test.loc_group.nunique()} unique locations')
    
    train = train_and_test.iloc[:len(train)]
    test = train_and_test.iloc[len(train):].drop(target, axis=1)
    
    return train, test

def cat_encode(train, test):
    # encoding the categorical feature in the train and test data set
    # using OneHotEncoder
    ohe = OneHotEncoder()
    train_encoded = ohe.fit_transform(train[['climateregions__climateregion']])
    test_encoded = ohe.transform(test[['climateregions__climateregion']])
    
    train = train.drop(['climateregions__climateregion'], axis=1)
    test = test.drop(['climateregions__climateregion'], axis=1)
    
    train_encoded = pd.DataFrame(train_encoded.toarray(), columns=ohe.get_feature_names_out(['climateregions__climateregion']))
    test_encoded = pd.DataFrame(test_encoded.toarray(), columns=ohe.get_feature_names_out(['climateregions__climateregion']))
    
    train = pd.concat([train, train_encoded], axis=1)
    test = pd.concat([test, test_encoded], axis=1)
    
    return train, test

    
def fill_na_rows(dataset):
    # Find the columns with missing values
    columns_with_missing_values = dataset.columns[dataset.isnull().any()].tolist()
    
    # Impute the missing values with the mean value of that column
    for col in columns_with_missing_values:
        dataset[col].fillna(dataset[col].mean(), inplace=True)
        
    return dataset

def create_new_feat(dataset):
#     dataset['year']=pd.DatetimeIndex(dataset['startdate']).year 
    dataset['month']=pd.DatetimeIndex(dataset['startdate']).month 
    dataset['day']=pd.DatetimeIndex(dataset['startdate']).day
#     dataset['season'] = dataset['month'].apply(lambda x: month_to_season[x])
    return dataset

def feature_engineering(origin_train, origin_test):
    train, test = origin_train, origin_test
    train = fill_na_rows(train)
    train = create_new_feat(train)
    test = create_new_feat(test)
    train, test = cat_encode(train, test)
    irrelevant_cols = ['index', 'startdate','contest-tmp2m-14d__tmp2m', 'climateregions__climateregion']
    features = [col for col in train.columns if col not in irrelevant_cols]
    #features = [col for col in train.columns]
    X = train[features]
    X_test = test[features]
    y = train['contest-tmp2m-14d__tmp2m']
    # Initialize the scaler
    #scaler = MinMaxScaler()

    # Fit the scaler to the train data
    #scaler.fit(X)

    # Transform the train data
    #X_train_scaled = scaler.transform(X)

    # Transform the test data
    #X_test_scaled = scaler.transform(X_test)
    
    return X, y, X_test

In [8]:
target="contest-tmp2m-14d__tmp2m"
cc_test_copy = cc_test.copy()

In [9]:
# Split the data into training and test sets
X, y, X_test = feature_engineering(cc_train.copy(), cc_test.copy())

In [10]:
def identify_correlated(df, threshold):
    corr_matrix = df.corr().abs()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    reduced_corr_matrix = corr_matrix.mask(mask)
    features_to_drop = [c for c in reduced_corr_matrix.columns if any(reduced_corr_matrix[c] > threshold)]
    return features_to_drop

In [11]:
# features_to_drop = identify_correlated(cc_train, .85)

In [12]:
features_to_drop = ['index',
 'nmme0-tmp2m-34w__cancm30',
 'nmme0-tmp2m-34w__cancm40',
 'nmme0-tmp2m-34w__ccsm30',
 'nmme0-tmp2m-34w__ccsm40',
 'nmme0-tmp2m-34w__cfsv20',
 'nmme0-tmp2m-34w__gfdlflora0',
 'nmme0-tmp2m-34w__gfdlflorb0',
 'nmme0-tmp2m-34w__gfdl0',
 'nmme0-tmp2m-34w__nasa0',
 'nmme0-tmp2m-34w__nmme0mean',
 'contest-wind-h10-14d__wind-hgt-10',
 'nmme-tmp2m-56w__cancm3',
 'nmme-tmp2m-56w__cancm4',
 'nmme-tmp2m-56w__ccsm3',
 'nmme-tmp2m-56w__ccsm4',
 'nmme-tmp2m-56w__cfsv2',
 'nmme-tmp2m-56w__gfdl',
 'nmme-tmp2m-56w__gfdlflora',
 'nmme-tmp2m-56w__gfdlflorb',
 'nmme-tmp2m-56w__nasa',
 'nmme-tmp2m-56w__nmmemean',
 'nmme-prate-34w__cancm3',
 'nmme-prate-34w__cancm4',
 'nmme-prate-34w__ccsm3',
 'nmme-prate-34w__ccsm4',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__gfdl',
 'nmme-prate-34w__gfdlflora',
 'nmme-prate-34w__gfdlflorb',
 'nmme-prate-34w__nasa',
 'nmme-prate-34w__nmmemean',
 'contest-wind-h100-14d__wind-hgt-100',
 'nmme0-prate-56w__cancm30',
 'nmme0-prate-56w__cancm40',
 'nmme0-prate-56w__ccsm30',
 'nmme0-prate-56w__ccsm40',
 'nmme0-prate-56w__cfsv20',
 'nmme0-prate-56w__gfdlflora0',
 'nmme0-prate-56w__gfdlflorb0',
 'nmme0-prate-56w__gfdl0',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-56w__nmme0mean',
 'nmme0-prate-34w__cancm30',
 'nmme0-prate-34w__cancm40',
 'nmme0-prate-34w__ccsm40',
 'nmme0-prate-34w__cfsv20',
 'nmme0-prate-34w__gfdlflora0',
 'nmme0-prate-34w__gfdlflorb0',
 'nmme0-prate-34w__nmme0mean',
 'contest-tmp2m-14d__tmp2m',
 'nmme-prate-56w__cancm3',
 'nmme-prate-56w__cancm4',
 'nmme-prate-56w__ccsm4',
 'nmme-prate-56w__cfsv2',
 'nmme-prate-56w__gfdl',
 'nmme-prate-56w__gfdlflora',
 'nmme-prate-56w__gfdlflorb',
 'contest-pres-sfc-gauss-14d__pres',
 'nmme-tmp2m-34w__cancm3',
 'nmme-tmp2m-34w__cancm4',
 'nmme-tmp2m-34w__ccsm3',
 'nmme-tmp2m-34w__ccsm4',
 'nmme-tmp2m-34w__cfsv2',
 'nmme-tmp2m-34w__gfdl',
 'nmme-tmp2m-34w__gfdlflora',
 'nmme-tmp2m-34w__gfdlflorb',
 'nmme-tmp2m-34w__nasa',
 'nmme-tmp2m-34w__nmmemean',
 'cancm30',
 'cancm40',
 'ccsm30',
 'ccsm40',
 'cfsv20',
 'gfdlflora0',
 'gfdlflorb0',
 'gfdl0',
 'nasa0',
 'nmme0mean',
 'wind-vwnd-250-2010-1',
 'wind-uwnd-250-2010-1',
 'mei__mei',
 'wind-hgt-850-2010-1',
 'sst-2010-1',
 'wind-hgt-500-2010-1',
 'icec-2010-1',
 'wind-uwnd-925-2010-1',
 'wind-hgt-10-2010-1',
 'wind-hgt-10-2010-2',
 'wind-hgt-100-2010-1']

In [13]:
remove_feature = ['index', 'contest-tmp2m-14d__tmp2m']
features_to_drop_v1 = [ele for ele in features_to_drop if ele not in remove_feature]
features_to_drop_v1

['nmme0-tmp2m-34w__cancm30',
 'nmme0-tmp2m-34w__cancm40',
 'nmme0-tmp2m-34w__ccsm30',
 'nmme0-tmp2m-34w__ccsm40',
 'nmme0-tmp2m-34w__cfsv20',
 'nmme0-tmp2m-34w__gfdlflora0',
 'nmme0-tmp2m-34w__gfdlflorb0',
 'nmme0-tmp2m-34w__gfdl0',
 'nmme0-tmp2m-34w__nasa0',
 'nmme0-tmp2m-34w__nmme0mean',
 'contest-wind-h10-14d__wind-hgt-10',
 'nmme-tmp2m-56w__cancm3',
 'nmme-tmp2m-56w__cancm4',
 'nmme-tmp2m-56w__ccsm3',
 'nmme-tmp2m-56w__ccsm4',
 'nmme-tmp2m-56w__cfsv2',
 'nmme-tmp2m-56w__gfdl',
 'nmme-tmp2m-56w__gfdlflora',
 'nmme-tmp2m-56w__gfdlflorb',
 'nmme-tmp2m-56w__nasa',
 'nmme-tmp2m-56w__nmmemean',
 'nmme-prate-34w__cancm3',
 'nmme-prate-34w__cancm4',
 'nmme-prate-34w__ccsm3',
 'nmme-prate-34w__ccsm4',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__gfdl',
 'nmme-prate-34w__gfdlflora',
 'nmme-prate-34w__gfdlflorb',
 'nmme-prate-34w__nasa',
 'nmme-prate-34w__nmmemean',
 'contest-wind-h100-14d__wind-hgt-100',
 'nmme0-prate-56w__cancm30',
 'nmme0-prate-56w__cancm40',
 'nmme0-prate-56w__ccsm30',
 'n

In [14]:
cc_train_reduced = pd.DataFrame(X.drop(features_to_drop_v1, axis=1))
cc_test_reduced = pd.DataFrame(X_test.drop(features_to_drop_v1, axis=1))

In [15]:
# X_train, X_test_tts, _train, y_test = train_test_split(cc_train_reduced, y, test_size=0.33, random_state=42)

In [16]:
X_train = cc_train_reduced
Y_train = y

In [17]:
nfolds = 5
folds = GroupKFold(n_splits = 5)
groups = np.array(X_train.month.values)
groups.shape

(400817,)

In [18]:
cat_cols = ["month", "day"]

In [19]:
params = {'iterations' : 12000,'learning_rate': 0.05,'cat_features' : cat_cols , 'loss_function':"RMSE", 'task_type':"GPU"}

In [20]:
X_train.shape

(400817, 171)

In [21]:
def rmse(actual, predicted):
    return mean_squared_error(actual, predicted, squared=False)

In [22]:
lgb_train_preds = np.zeros(X_train.shape[0])
lgb_test_preds = np.zeros(cc_test_reduced.shape[0])
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train.values, X_train.values, groups)):
    print('----')
    print("fold n°{}".format(fold_))
    x_train,y_train = X_train.iloc[trn_idx], Y_train[trn_idx]
    x_val,y_val = X_train.iloc[val_idx], Y_train[val_idx]
    lgb_model = CatBoostRegressor(**params).fit(x_train, y_train, 
                                                  eval_set=[(x_val, y_val)],
                                                  early_stopping_rounds=200, verbose = 100)
    test_oof_preds = lgb_model.predict(cc_test_reduced)
    train_oof_preds = lgb_model.predict(x_val)
    lgb_train_preds[val_idx] = train_oof_preds
    lgb_test_preds += test_oof_preds / nfolds
    print(": CB - RMSE = {}".format(rmse(y_val, train_oof_preds)))
print(": CB - RMSE = {}".format(rmse(Y_train, lgb_train_preds)))

----
fold n°0
0:	learn: 8.7333753	test: 11.3525438	best: 11.3525438 (0)	total: 31.2ms	remaining: 6m 14s
100:	learn: 1.6192792	test: 1.9599899	best: 1.9599899 (100)	total: 2.61s	remaining: 5m 7s
200:	learn: 1.3156396	test: 1.6420288	best: 1.6420288 (200)	total: 5.17s	remaining: 5m 3s
300:	learn: 1.1547980	test: 1.5062422	best: 1.5062422 (300)	total: 8.97s	remaining: 5m 48s
400:	learn: 1.0427479	test: 1.4221681	best: 1.4219201 (399)	total: 11.5s	remaining: 5m 33s
500:	learn: 0.9601756	test: 1.3652953	best: 1.3652953 (500)	total: 14.1s	remaining: 5m 23s
600:	learn: 0.8974332	test: 1.3230720	best: 1.3230720 (600)	total: 16.7s	remaining: 5m 15s
700:	learn: 0.8441160	test: 1.2888320	best: 1.2888320 (700)	total: 19.6s	remaining: 5m 15s
800:	learn: 0.8017150	test: 1.2636646	best: 1.2636646 (800)	total: 22.1s	remaining: 5m 9s
900:	learn: 0.7681273	test: 1.2395005	best: 1.2395005 (900)	total: 24.6s	remaining: 5m 3s
1000:	learn: 0.7370785	test: 1.2214014	best: 1.2214014 (1000)	total: 27.2s	remain

In [23]:
cc_sample[target] = lgb_test_preds
cc_sample.to_csv('/kaggle/working/submission_drop_year_add_season.csv', index = False)

In [24]:
#weight for month